###  Построение Look-alike модели для целевой аудитории раздела веб-сайта

Используются различные подходы:
- Логистическая регрессия
- Naive Bayes
- текущий подход, рассмотренный в Wiki[https://wiki.tcsbank.ru/pages/viewpage.action?pageId=176096365].

Сравнение методов производится по метрике AUC ROC.

** Модификация - все через Hive, в Spark только само обучение и вывод результатов. **

In [12]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime

sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "8g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

connection = pyhs2.connect(host='ds-hadoop-cs01p.tcsbank.ru',
                           port=10000,
                           authMechanism='PLAIN',
                           user='kposminin',
                           password='')
c = connection.cursor()

NameError: name 'pyhs2' is not defined

In [9]:
#Params
start = datetime.datetime.now()
target_urls =['raiffeisen.ru/retail/cards/credit']#['avito.ru/moskva'] #
exclude_urls = target_urls + ['raiffeisen.ru'] #['avito.ru']  #

#source_table_name = 'user_kposminin.access_log_sample'
#train_start_date, train_end_date = '2016-06-01', '2016-06-02' 
#test_date = '2016-06-30'

source_table_name = 'user_kposminin.access_log_sample5'
train_start_date, train_end_date = '2016-07-07', '2016-07-07'
test_date = '2016-07-14'



In [11]:
#Hive queries

#based on urlp column

calc_id = 21 #int(hc.sql('select max(id) from user_kposminin.calcs').collect()[0][0]) + 1

target_expression = '('+'or'.join(' url like "%' + u + '%" ' for u in target_urls)+')'
exclude_expression = 'not ('+'or'.join(' urlp like "%' + u + '%" ' for u in exclude_urls)+')'

update_calcs_table_query = '''
insert into user_kposminin.calcs values(
    {calc_id},
    {date},
    "Look alike model comparison","Comparison of NaiveBayes, LogisticRegression and Current approach",
    "train_start_date: {train_start_date}, train_end_date:train_end_date,
        test_date: {test_date}, source_table_name: {source_table_name},
        target_urls: {target_urls}, exclude_urls: {exclude_urls}"
    )
'''.format(
    calc_id = calc_id, 
    train_start_date = train_start_date, 
    train_end_date = train_end_date, 
    test_date = test_date,
    source_table_name = source_table_name,
    target_urls = str(target_urls),    
    exclude_urls = str(exclude_urls),
    date = str(datetime.datetime.now().date())
)


create_tables_in_hive_query = '''
drop table if exists user_kposminin.urls_w_levels_train{calc_id};

create table user_kposminin.urls_w_levels_train{calc_id} as
select
    a.id as cookie
    ,concat(a.id, "-", a.ymd) as object_id
    ,a.ymd
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)', 1) as domain
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?', 2) lev0
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?', 3) lev1
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?/?([^/]*)?', 4) lev2
    ,a.url
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)', 1) as ref_domain
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?', 2) ref_lev0
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?', 3) ref_lev1
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?/?([^/]*)?', 4) ref_lev2
    ,a.referrer
    ,a.timestamp
    from 
    (
       select b.*
        from
           (select
              al.*,
              count(*) over (partition by url) as url_count
           from {source_table_name} al
              where ymd between "{train_start_date}" and "{train_end_date}"
           ) b
       where b.url_count > 50
    ) a
;

drop table if exists user_kposminin.urls_w_levels_test{calc_id} ;

create table user_kposminin.urls_w_levels_test{calc_id} as
select
    a.id as cookie
    ,concat(a.id, "-", a.ymd) as object_id
    ,a.ymd
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)', 1) as domain
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?', 2) lev0
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?', 3) lev1
    ,regexp_extract(regexp_extract(a.url, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?/?([^/]*)?', 4) lev2
    ,a.url
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)', 1) as ref_domain
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?', 2) ref_lev0
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?', 3) ref_lev1
    ,regexp_extract(regexp_extract(a.referrer, "([^\?]*)", 0), '^([^/]*)/?([^/]*)?/?([^/]*)?/?([^/]*)?', 4) ref_lev2
    ,a.referrer
    ,a.timestamp
    from 
    (
       select b.*
        from
           (select
              al.*,
              count(*) over (partition by url) as url_count
           from {source_table_name} al
              where ymd = "{test_date}"
           ) b
       where b.url_count > 50
    ) a
;

drop table if exists user_kposminin.user_urlp_train{calc_id};

create table user_kposminin.user_urlp_train{calc_id} as 
   select 
        concat(cookie, 0) as cookie,
        domain as urlp    
    from user_kposminin.urls_w_levels_train{calc_id}
    union all
    select
        concat(cookie, 0) as cookie,
        concat(domain,'[0]',lev0) as urlp
     from user_kposminin.urls_w_levels_train{calc_id}
    union all
    select 
        concat(cookie, 0) as cookie,
        concat(domain,'[1]',lev1) as urlp
    from user_kposminin.urls_w_levels_train{calc_id}
    union all
    select 
        concat(cookie, 0) as cookie,
        concat(domain,'[2]',lev2) as urlp
    from user_kposminin.urls_w_levels_train{calc_id}
;

drop table if exists user_kposminin.user_urlp_test{calc_id};

create table user_kposminin.user_urlp_test{calc_id} as 
   select 
        concat(cookie, 1) as cookie,
        domain as urlp    
    from user_kposminin.urls_w_levels_test{calc_id}
    union all
    select
        concat(cookie, 1) as cookie,
        concat(domain,'[0]',lev0) as urlp
     from user_kposminin.urls_w_levels_test{calc_id}
    union all
    select 
        concat(cookie, 1) as cookie,
        concat(domain,'[1]',lev1) as urlp
    from user_kposminin.urls_w_levels_test{calc_id}
    union all
    select 
        concat(cookie, 1) as cookie,
        concat(domain,'[2]',lev2) as urlp
    from user_kposminin.urls_w_levels_test{calc_id}   
;

drop table if exists user_kposminin.user_train{calc_id};

create table user_kposminin.user_train{calc_id} as
select
   concat(id, 0)  as cookie,
   max(case when {target_expression} then 1 else 0 end) as label
from 
   {source_table_name} al
   where ymd between "{train_start_date}" and "{train_end_date}"
group by id;



drop table if exists user_kposminin.user_test{calc_id};

create table user_kposminin.user_test{calc_id} as
select
   concat(id, 1) as cookie,
   max(case when {target_expression} then 1 else 0 end) as label
from 
   {source_table_name} al
   where ymd = "{test_date}"
group by id;


drop table if exists user_kposminin.urlp_score_train{calc_id};

create table user_kposminin.urlp_score_train{calc_id} as
select
    urlp,
    log((positives + 0.5) / (total - positives + 0.5)) as score
from
    (select
        urlp,
        sum(label) as positives,
        count(cookie) as total
    from
        (select distinct
            a.urlp,
            a.cookie,
            b.label
        from 
            (select * 
             from user_kposminin.user_urlp_train{calc_id}
             where {exclude_expression}
            ) a
        left join user_kposminin.user_train{calc_id} b 
        on a.cookie = b.cookie
        ) c
    group by urlp
    ) d
where
    total > 20
    or positives > 1
;


drop table if exists user_kposminin.user_score_test{calc_id};

create table user_kposminin.user_score_test{calc_id} as
select
    cs.cookie,
    cs.score,
    i.label
from
    (select 
        u.cookie,
        max(s.score) as score
    from 
        user_kposminin.user_urlp_test{calc_id} u
    join user_kposminin.urlp_score_train{calc_id} s
    on u.urlp = s.urlp
    group by u.cookie) cs
join user_kposminin.user_test{calc_id} i
on i.cookie = cs.cookie;

drop table if exists user_kposminin.user_score_test_exper{calc_id};

create table user_kposminin.user_score_test_exper{calc_id} as
select
    cs.cookie,
    cs.max_score,
    cs.sum_score,
    cs.avg_score,
    cs.scores_list,
    i.label    
from
    (select 
        u.cookie,
        max(s.score) as max_score,
        sum(s.score) as sum_score,
        avg(s.score) as avg_score,
        sort_array(collect_list(s.score)) as scores_list
    from 
        user_kposminin.user_urlp_test{calc_id} u
    join user_kposminin.urlp_score_train{calc_id} s
    on u.urlp = s.urlp
    group by u.cookie) cs
join user_kposminin.user_test{calc_id} i
on i.cookie = cs.cookie'''.format(
    calc_id = calc_id, 
    train_start_date = train_start_date, 
    train_end_date = train_end_date, 
    test_date = test_date,
    source_table_name = source_table_name,
    target_expression = target_expression,
    exclude_expression = exclude_expression
)

train_labeledpoint_query = '''
select
    u.label,
    cu.url_list
from
   (select
      cookie,
      collect_list(urlp) as url_list
   from 
      user_kposminin.user_urlp_train{calc_id}
   where {exclude_expression}
   group by cookie) cu
join user_kposminin.user_train{calc_id} u
on cu.cookie = u.cookie
'''.format(calc_id = calc_id, exclude_expression = exclude_expression)

test_labeledpoint_query = '''
select
    u.label,
    cu.url_list
from
   (select
      cookie,
      collect_list(urlp) as url_list
   from 
      user_kposminin.user_urlp_test{calc_id}
   where {exclude_expression}
   group by cookie) cu
join user_kposminin.user_test{calc_id} u
on cu.cookie = u.cookie
'''.format(calc_id = calc_id, exclude_expression = exclude_expression)

current_approach_results_query = '''
select
    *
from 
    user_kposminin.user_score_test_exper{calc_id}
'''.format(calc_id = calc_id)

In [8]:
print(update_calcs_table_query)


insert into user_kposminin.calcs values(
    19,
    2016-08-25,
    "Look alike model comparison","Comparison of NaiveBayes, LogisticRegression and Current approach",
    "train_start_date: 2016-07-07, train_end_date:train_end_date,
        test_date: 2016-07-14, source_table_name: user_kposminin.access_log_sample5,
        target_urls: ['raiffeisen.ru/retail/cards/credit'], exclude_urls: ['raiffeisen.ru/retail/cards/credit', 'raiffeisen.ru']"
    )



In [5]:
# Make calculations and create tables in Hive

#hc.sql(update_calcs_table_query)
#for q in create_tables_in_hive_query.split(';'):
#    print(create_tables_in_hive_query.split(';').index(q))
#    hc.sql(q)

In [ ]:
#Load train and test data to Spark

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel

tf = HashingTF(numFeatures = 10 ** 6)

#transform urls (as Bag of Words) into features and form features with labels
train_data = hc.sql(train_labeledpoint_query).rdd.map(lambda r: LabeledPoint(r.label,tf.transform(r.url_list)))
train_data.cache()

test_data = hc.sql(test_labeledpoint_query).rdd.map(lambda r: LabeledPoint(r.label,tf.transform(r.url_list)))

In [ ]:
#Train NaiveBayes model

modelNB = NaiveBayes.train(train_data)

def predict_proba_NB(f,model):
    import numpy as np
    '''
    Naive Bayes model prediction with probability. f is features [Sparse] vector. model is mllib.NaiveBayesModel.
    Function selects winning class with it probability.
    Output: tuple with model selected class number as first element (type int) and it probability as second (type float).
    '''
    logp = [[i,f.dot(model.theta[i]) + model.pi[i]] for i in range(len(model.theta))] # classes with log probabilities
    wi = sorted(logp, key = lambda e:  - e[1])[0][0] #winning index
    prob = 1./sum([np.exp(e[1] - logp[wi][1]) for e in logp]) #winning class probability
    return wi, prob

def predict_proba_NB_2(f, model):
    import numpy as np
    '''
    Naive Bayes model prediction with probability for 2-class classification.
    f is features [Sparse] vector. model is mllib.NaiveBayesModel.
    Output: probability of class 1 (type float).
    '''
    if len(model.theta) != 2:
        print('Model is NOT a 2-class classifier')
        return None
    logp = [f.dot(model.theta[i]) + model.pi[i] for i in range(2)]    
    return 1./(1. + np.exp(logp[0] - logp[1]))


In [ ]:
#LogisticRegression model

modelLR = LogisticRegressionWithSGD.train(train_data)
modelLR.clearThreshold()

In [ ]:
# 3. Current approach (results only). All calculations in Hive

ca_res = hc.sql(current_approach_results_query).toPandas()

In [ ]:
#Testing result

df_test = test_data.map( lambda lp: pyspark.sql.Row(
        Label = lp.label,
        NaiveBayes = float(predict_proba_NB_2(lp.features, modelNB)),
        LogisticRegression = float(modelLR.predict(lp.features))
    )).toDF().toPandas()


In [ ]:
#Build AUCROC metric and print results

import sklearn
AUCROC = []
for c in df_test.columns:
    if c!= 'Label':
        AUCROC.append([c,sklearn.metrics.roc_auc_score(df_test['Label'],df_test[c])])
for c in [c for c in ca_res.columns if not c in [u'cookie', u'scores_list', u'label']]:
    AUCROC.append(['CurApp_' + c, sklearn.metrics.roc_auc_score(ca_res['label'], ca_res[c])])
for n in [2,3,5,7,10,15,20]:
    AUCROC.append(['CurApp_Top' + str(n), sklearn.metrics.roc_auc_score(
        ca_res['label'], [sum(r[-n:])/max(len(r[-n:]),1) for r in ca_res[u'scores_list'].values]
    )])
        
print('\nMethods AUCROC performance on test sample ({0:.0f} samples with {1:.0f} positives):\n\n'.format(
        df_test.size,df_test['Label'].sum()) +'\n'.join(['{0:<30}{1:.5f}'.format(k,v) for (k,v) in AUCROC])
)


In [ ]:
print('Time of work {0}'.format(datetime.datetime.now() - start))